# Installation

In [5]:
!pip install diffusers --upgrade

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [6]:
!pip install invisible_watermark transformers accelerate safetensors

In [7]:
import cv2
import os
import numpy as np
import torch
from diffusers import DiffusionPipeline
from tqdm import tqdm
import pickle
import time

# Set the image files and the prompt file path

In [8]:
promptFilePath = '' # set the file path to the prompt file

In [9]:
promptList = []

height = 512
width = 512

with open(promptFilePath, "r") as file:
    for line in file:
        promptList.append(line.strip())

In [10]:
pipe = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", 
                                         torch_dtype=torch.float16, 
                                         use_safetensors=True, 
                                         height=height,
                                         width=width,
                                         variant="fp16")
pipe.to("cuda")

model_index.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]

tokenizer/merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

text_encoder/config.json:   0%|          | 0.00/565 [00:00<?, ?B/s]

model.fp16.safetensors:   0%|          | 0.00/246M [00:00<?, ?B/s]

...

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

vae/config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

Keyword arguments {'height': 512, 'width': 512} are not expected by StableDiffusionXLPipeline and will be ignored.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

StableDiffusionXLPipeline {
  "_class_name": "StableDiffusionXLPipeline",
  "_diffusers_version": "0.32.1",
  "_name_or_path": "stabilityai/stable-diffusion-xl-base-1.0",
  "feature_extractor": [
    null,
    null
  ],
  "force_zeros_for_empty_prompt": true,
  "image_encoder": [
    null,
    null
  ],
  "scheduler": [
    "diffusers",
    "EulerDiscreteScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "text_encoder_2": [
    "transformers",
    "CLIPTextModelWithProjection"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "tokenizer_2": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [ ]:
timeList = []
imageListOne = []
imageListTwo = []

for i in tqdm(range(len(promptList))):
    print(i)
    start = time.time()
    prompt = promptList[i].split(' : ')
    #print(prompt)
    # if using torch < 2.0
    # pipe.enable_xformers_memory_efficient_attention()    
    images = pipe(prompt=prompt).images
    end = time.time()
    # append the first of generated images to list one
    imageListOne.append(images[0])
    # append the second of generated images to list two
    imageListTwo.append(images[1])
    print('image1 shape: ', images[0])
    print('image2 shape: ', images[1])
    timeList.append(end - start)
    #break

  0%|          | 0/200 [00:00<?, ?it/s]

0


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/200 [01:25<?, ?it/s]

image1 shape:  <PIL.Image.Image image mode=RGB size=1024x1024 at 0x7C89E3918E80>
image2 shape:  <PIL.Image.Image image mode=RGB size=1024x1024 at 0x7C89E39191E0>


In [ ]:
print('timeList: ', len(timeList))
print('imageListOne: ', len(imageListOne))
print('imageListTwo: ', len(imageListTwo))

# Save imageList as a pickle file

In [ ]:
generated_image_file_path_one = '' # declare the file path to save the generated images as pkl file
generated_image_file_path_two = '' # declare the file path to save the generated images as pkl file


with open(generated_image_file_path_one, 'wb') as f:
    pickle.dump(imageListOne, f)

with open(generated_image_file_path_two, 'wb') as f:
    pickle.dump(imageListTwo, f)

# Save timeList as a pickle file

In [ ]:
generated_time_path = '' # declare the file path to save the time sheet for image generation as a pkl file

with open(generated_time_path, 'wb') as f:
    pickle.dump(timeList, f)

In [ ]:
#references:
#https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0
#changing the dimensions of the images generated using stable diffusion:
#https://huggingface.co/docs/diffusers/main/en/using-diffusers/sdxl